<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


read dev dataset

In [ ]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

read train dataset

In [ ]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

## **Tokenize Data**

install packages

In [ ]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import the package
import stanza

     |████████████████████████████████| 432 kB 5.4 MB/s 
     |████████████████████████████████| 174 kB 54.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=0155812afc46baffa28bfea6005de94a722f018506d957a421eddc6f692bb141
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-26 07:20:44 INFO: Downloading default packages for language: fa (Persian)...


2022-01-26 07:20:50 INFO: Finished downloading models and saved to /root/stanza_resources.


create models

In [ ]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-26 07:46:57 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-26 07:46:57 INFO: Use device: cpu
2022-01-26 07:46:57 INFO: Loading: tokenize
2022-01-26 07:46:57 INFO: Loading: mwt
2022-01-26 07:46:57 INFO: Loading: pos


Building a Persian pipeline...


2022-01-26 07:46:58 INFO: Loading: lemma
2022-01-26 07:46:58 INFO: Loading: depparse
2022-01-26 07:46:59 INFO: Done loading processors!


find tokens of dataframe and save in array

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


save tokens array to csv file

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

convert tokens array to dataframe 

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

save dataframe to csv file

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

# **Word embedding**

In [225]:
#read dataframe csv file
# tokens_dataframe=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_df.csv")
# tokens_dataframe=tokens_dataframe.drop("Unnamed: 0",axis=1)
# #tokens_dataframe[52000:52050]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342

In [226]:
# arr=tokens_dataframe.to_numpy()
# tokens_arr=[]
# for i in arr :
#   sentense=[]
#   for word in i:
#     if type(word) is str:
#       sentense.append(word)
#   tokens_arr.append(sentense)

read from array csv file (smaller size)

تو از اینجا ران کن(فایلش رو تلگرام فرستادم برات)

In [227]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

In [228]:
# from text_normalizer.text_normalizer_collection_library import normalizer
from types import new_class
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i:
    sent.append(j)
  tokens_arr.append(sent)

In [229]:
for i in tokens_arr[:10]:
  print(i)

['واقعا,حیف,وقت,که,بنویسم,سرویس,دهیتون,شده,افتضاح']
['قرار,بود,۱,ساعته,برسه,ولی,نیم,ساعت,زودتر,از,موقع,رسید,،,شما,ببین,چقدرررررررررررر,پلاک,خفنههههه,،,من,سالها,ست,مشتریشونم,و,سالها,ست,مزه,بهشت,میده,غذاشون']
['قیمت,این,مدل,اصلا,با,کیفیتش,سازگاری,نداره,،,فقط,ظاهر,فریبنده,داره,،,پرش,میکنن,کالباس,و,قارچ']
['عالللی,بود,همه,چه,درست,و,به,اندازه,و,کیفیت,خوب,،,امیداورم,همیشه,کیفیتتون,خوب,باشه,ما,مشتری,همیشگی,بشیم']
['شیرینی,وانیلی,فقط,یک,مدل,بود,.']
['بدترین,پیتزایی,که,تا,به,حال,خورده,بودم']
['از,همه,لحاظ,عالی,ممنون,م']
['کیفیت,غذا,متوسط,رو,به,پایین,بود,انگار,داخل,یه,رستوران,معمولی,غذا,خوردی,درحالی,که,امتیاز,رستوران,در,اسنپ,فود,۴٫۳,بود']
['همه,اقلام,تازه,و,به,روز,وخیلیییییی,سریع,بدستم,رسید,واقعا,متشکرم']
['همه,چی,خوب,ولی,هات,داگ,دور,ش,کلا,سوخته,بود,و,داخلش,خام,بود,!,!,!,!']
